In [3]:
from IPython import display
import ultralytics
from ultralytics import YOLO
import cv2
import os
import re
import glob
import shutil
import subprocess
framecounter = 0
seqlength = int

Convert video into a folder of jpg frames

In [2]:
def convert_mp4_to_frames(video_path, output_folder):
    """
    Converts an MP4 video into a folder of JPG frames.

    Args:
        video_path (str): Path to the input MP4 video.
        output_folder (str): Path to the folder where frames will be saved.
    """
    global seqlength
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    if not video_capture.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    frame_count = 1

    while True:
        ret, frame = video_capture.read()

        # Break the loop when no frames are left to read
        if not ret:
            break

        # Define the frame's output filename
        frame_filename = os.path.join(output_folder, f"{frame_count:04d}.jpg")

        # Save the current frame as a JPG file
        cv2.imwrite(frame_filename, frame)
        print(f"Saved frame {frame_count} to {frame_filename}")

        frame_count += 1
    seqlength = frame_count

    # Release the video capture object
    video_capture.release()
    print(f"Conversion complete! Total frames extracted: {frame_count}")

# Example usage:
video_path = "C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/sample_nl_match.mp4"
output_folder = "C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq"
convert_mp4_to_frames(video_path, output_folder)

Saved frame 1 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0001.jpg
Saved frame 2 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0002.jpg
Saved frame 3 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0003.jpg
Saved frame 4 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0004.jpg
Saved frame 5 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0005.jpg
Saved frame 6 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0006.jpg
Saved frame 7 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0007.jpg
Saved frame 8 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0008.jpg
Saved frame 9 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0009.jpg
Saved frame 10 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0010.jpg
Saved frame 11 to C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/originalseq\0011.jpg
Saved fr

In [15]:
model = YOLO("C:/Studie/BEP/BEP_Football_tracking/YOLO_results/Ball10/best.pt")

In [16]:
# Run inference on 'bus.jpg' with arguments
results = model.predict("C:/Studie/BEP/BEP_Football_tracking/sample_nl_match.mp4", save=False, imgsz=1280, conf=0.60, show=True, save_txt= True, max_det = 1, project = 'C:/Studie/BEP/BEP_Football_tracking/hybrid_workflow/YOLO_ball')



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 43.9ms
video 1/1 (frame 2/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 31.9ms
video 1/1 (frame 3/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 31.8ms
video 1/1 (frame 4/727) C:\Studie\BEP\BEP_Football_tracking\sample_nl_match.mp4: 736x1280 (no detections), 31.9ms
vi

Code that gets rid of first variable in yolo text predictions so it is ready for prompting(assuming a 1920 by 1080 resolution
YOLO uses a center based relative format x,y(of center) width hight between 0 and 1
Samurai uses as promp (x,y of the top left, widht height) all in pixels

In [30]:
output_folder = r'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\prompts'

In [31]:
def convert_filename(yolo_file):
    # Use a regular expression to find the number at the end of the filename, before the extension
    match = re.search(r'(\d+)(?=\.[a-zA-Z]+$)', yolo_file)

    if match:
        # Extract the number (it can have multiple digits)
        number = match.group(1)

        # Return the new filename using only the extracted number and .txt extension
        return f"{number}.txt"
    else:
        raise ValueError("Filename does not have a number at the end")

def yolo_to_samurai(yolo_file, output_folder, resolution=(1920, 1080)):
    # Define the target directory
    target_directory = output_folder

    # Convert YOLO filename to Samurai format filename
    output_filename = convert_filename(yolo_file)

    # Combine the target directory and the new filename to create the full path
    output_file = os.path.join(target_directory, output_filename)

    # Ensure the target directory exists
    os.makedirs(target_directory, exist_ok=True)

    with open(yolo_file, 'r') as file:
        yolo_lines = file.readlines()

    samurai_lines = []
    for line in yolo_lines:
        # Split YOLO line into components
        values = line.split()
        if len(values) != 5:
            continue  # Skip malformed lines

        _, x_center_rel, y_center_rel, width_rel, height_rel = map(float, values)

        # Convert relative values to pixel values
        x_center = x_center_rel * resolution[0]
        y_center = y_center_rel * resolution[1]
        width = width_rel * resolution[0]
        height = height_rel * resolution[1]

        # Calculate top-left corner coordinates
        x_top_left = x_center - width / 2
        y_top_left = y_center - height / 2

        # Format as Samurai bounding box
        samurai_line = f"{int(round(x_top_left))}, {int(round(y_top_left))}, {int(round(width))}, {int(round(height))}\n"
        samurai_lines.append(samurai_line)

    # Write Samurai format to the specified output file path
    with open(output_file, 'w') as file:
        file.writelines(samurai_lines)


In [32]:
# Function to process all txt files in a directory
def process_all_files(input_folder, output_folder):
    # Find all .txt files in the directory
    yolo_files = glob.glob(os.path.join(input_folder, "*.txt"))

    for yolo_file in yolo_files:
        yolo_to_samurai(yolo_file, output_folder)



In [33]:
# Example usage
input_folder = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\YOLO_ball\predict2\labels'
output_folder = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\prompts'

process_all_files(input_folder, output_folder)

Initialization of a count variable that counts the current frame we are on, adds al the frames into the final sequence until a prompt is found. Remove the frames that are transfered from the original sequence

In [11]:
def copy_jpg_images(input_dir, output_dir):
    # Check if input directory exists
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"The input directory {input_dir} does not exist.")

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all files in the input directory
    for filename in os.listdir(input_dir):
        input_file_path = os.path.join(input_dir, filename)

        # Only process files (skip directories)
        if os.path.isfile(input_file_path):
            # Check if the file is a .jpg file
            if filename.lower().endswith('.jpg'):
                # Create the output file path
                output_file_path = os.path.join(output_dir, filename)
                # Copy the .jpg file to the output directory
                shutil.copy(input_file_path, output_file_path)
                print(f"Copied: {filename}")




In [14]:

samurai_running = False

In [12]:
frame = 1
samurai_running = False

def find_initial_prompt(originalseq_path, output_path, prompts_path):
    trackedseqvis_path = output_path
    global frame
    global samurai_running
    global framecounter

    # Create the output directory if it does not exist
    os.makedirs(trackedseqvis_path, exist_ok=True)

    # List and sort the text files in the prompts folder
    txt_files = sorted(os.listdir(prompts_path), key=lambda x: int(os.path.splitext(x)[0]))
    txt_frame_numbers = [int(os.path.splitext(file)[0]) for file in txt_files]

    while True:
        # Match frames with prompt text files
        if frame in txt_frame_numbers:
            samurai_running = True
            break
        else:
            # If the frame does not match, copy the corresponding image to trackedseqvis
            image_name = f"{frame:04d}.jpg"  # This assumes filename

            image_path = os.path.join(originalseq_path, image_name)

            if os.path.exists(image_path):

                # Remove the image from the original sequence
                os.remove(image_path)
            else:
                print(f"Image {image_name} not found in {originalseq_path}.")

        frame += 1
        framecounter = framecounter + frame_num




    print(f"YOLO Prompt found. Initializing Samurai: {samurai_running}")

In [15]:
originalseq = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\originalseq'
prompts_path = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow\prompts'
output_folder  = 'C:\Studie\BEP\BEP_Football_tracking\hybrid_workflow/tracked_seq'
copy_jpg_images(originalseq, output_folder)
find_initial_prompt(originalseq, output_folder, prompts_path)


Copied: 0078.jpg
Copied: 0079.jpg
Copied: 0080.jpg
Copied: 0081.jpg
Copied: 0082.jpg
Copied: 0083.jpg
Copied: 0084.jpg
Copied: 0085.jpg
Copied: 0086.jpg
Copied: 0087.jpg
Copied: 0088.jpg
Copied: 0089.jpg
Copied: 0090.jpg
Copied: 0091.jpg
Copied: 0092.jpg
Copied: 0093.jpg
Copied: 0094.jpg
Copied: 0095.jpg
Copied: 0096.jpg
Copied: 0097.jpg
Copied: 0098.jpg
Copied: 0099.jpg
Copied: 0100.jpg
Copied: 0101.jpg
Copied: 0102.jpg
Copied: 0103.jpg
Copied: 0104.jpg
Copied: 0105.jpg
Copied: 0106.jpg
Copied: 0107.jpg
Copied: 0108.jpg
Copied: 0109.jpg
Copied: 0110.jpg
Copied: 0111.jpg
Copied: 0112.jpg
Copied: 0113.jpg
Copied: 0114.jpg
Copied: 0115.jpg
Copied: 0116.jpg
Copied: 0117.jpg
Copied: 0118.jpg
Copied: 0119.jpg
Copied: 0120.jpg
Copied: 0121.jpg
Copied: 0122.jpg
Copied: 0123.jpg
Copied: 0124.jpg
Copied: 0125.jpg
Copied: 0126.jpg
Copied: 0127.jpg
Copied: 0128.jpg
Copied: 0129.jpg
Copied: 0130.jpg
Copied: 0131.jpg
Copied: 0132.jpg
Copied: 0133.jpg
Copied: 0134.jpg
Copied: 0135.jpg
Copied: 0136.j

NameError: name 'frame_num' is not defined

Adjust working directory to samurai wroking directory

A function that takes a prompt and the original seq as input and runs samurai until a wrongfull bounding box is found

In [69]:
os.chdir("C:\Studie\BEP\BEP_Football_tracking\BEP_football_tracking\Samurai\samurai")
print(os.getcwd())  # Confirm the new working directory

C:\Studie\BEP\BEP_Football_tracking\BEP_football_tracking\Samurai\samurai


In [70]:
def run_script(sequence_path, prompt_no):
    global framecounter
    # Define the command with substitutions
    sequence_path = os.path.join(sequence_path, 'originalseq')
    prompts_path = os.path.join(prompts_path, 'prompts')
    command = f'python scripts/demo.py --video_path {sequence_path} --txt_path {prompt_no}'

    # Run the command using subprocess
    subprocess.run(command, shell=True, check=True)

In [71]:
# Example usage:
sequence_path = 'C:/Studie/BEP/BEP_Football_tracking/hybrid_workflow/originalseq'
prompt_no = 'C:/Studie/BEP/BEP_Football_tracking/hybrid_workflow/prompts/18.txt'

run_script(sequence_path, prompt_no)


In [72]:
def get_stopframeno(directory):
    # Get all .txt files in the directory
    txt_files = [f for f in os.listdir(directory) if f.endswith('.txt')]

    # If there are no .txt files, return None or raise an error
    if not txt_files:
        return None

    # Sort the list of files alphabetically (or by modification time if you prefer)
    txt_files.sort()  # Alphabetically

    # Get the last file
    last_file = txt_files[-1]

    # Remove the .txt extension and convert the result to an integer
    file_name_without_extension = last_file[:-4]


    try:
        return int(file_name_without_extension)
    except ValueError:
        # If the filename cannot be converted to an integer, return None
        return None




In [79]:
# Example usage
directory = 'C:\Studie\BEP\BEP_Football_tracking\BEP_football_tracking\Samurai\stopframes'  # Change this to your actual directory path
frames_passed = get_stopframeno(directory) - 2
framecounter = framecounter + frames_passed

50


In [89]:
def find_nearest_higher_int_prompts(framecounter: int, folder_path: str) -> int:
    """
    Finds the nearest integer higher than `framecounter` from the filenames
    in the given folder.

    Args:
        framecounter (int): The reference integer.
        folder_path (str): The path to the folder containing text files with integer names.

    Returns:
        int: The nearest higher integer, or None if no such integer exists.
    """
    try:
        # List all files in the directory
        files = os.listdir(folder_path)

        # Extract integers from filenames
        integers = []
        for file in files:
            try:
                # Attempt to convert the filename (without extension) to an integer
                number = int(os.path.splitext(file)[0])
                integers.append(number)
            except ValueError:
                # Ignore files with non-integer names
                continue

        # Filter integers that are higher than framecounter
        higher_integers = [num for num in integers if num >= framecounter]

        # Find the nearest higher integer
        if higher_integers:
            return min(higher_integers)
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


Make a function that uses the frame counter to find the  next suitable prompt

In [100]:
result = find_nearest_higher_int_prompts(framecounter, prompts_path)
print(result)
framecounter = result

78


In [101]:
def remove_frames_below_framecounter(folder_path, framecounter):
    """
    Remove all frames in the folder whose number is less than framecounter.

    Parameters:
        folder_path (str): Path to the folder containing the frames.
        framecounter (int): Threshold frame number. Files with numbers below this will be removed.
    """
    try:
        # List all files in the folder
        files = os.listdir(folder_path)

        for file in files:
            # Ensure the file has a .jpg extension
            if file.endswith('.jpg'):
                # Extract the numeric part of the filename
                try:
                    frame_number = int(file.split('.')[0])
                except ValueError:
                    # Skip files that don't follow the naming convention
                    continue

                # Check if the frame number is below the framecounter
                if frame_number < framecounter:
                    file_path = os.path.join(folder_path, file)
                    os.remove(file_path)  # Remove the file
                    print(f"Removed: {file}")

    except Exception as e:
        print(f"An error occurred: {e}")



In [102]:
remove_frames_below_framecounter(sequence_path, framecounter)

Removed: 0018.jpg
Removed: 0019.jpg
Removed: 0020.jpg
Removed: 0021.jpg
Removed: 0022.jpg
Removed: 0023.jpg
Removed: 0024.jpg
Removed: 0025.jpg
Removed: 0026.jpg
Removed: 0027.jpg
Removed: 0028.jpg
Removed: 0029.jpg
Removed: 0030.jpg
Removed: 0031.jpg
Removed: 0032.jpg
Removed: 0033.jpg
Removed: 0034.jpg
Removed: 0035.jpg
Removed: 0036.jpg
Removed: 0037.jpg
Removed: 0038.jpg
Removed: 0039.jpg
Removed: 0040.jpg
Removed: 0041.jpg
Removed: 0042.jpg
Removed: 0043.jpg
Removed: 0044.jpg
Removed: 0045.jpg
Removed: 0046.jpg
Removed: 0047.jpg
Removed: 0048.jpg
Removed: 0049.jpg
Removed: 0050.jpg
Removed: 0051.jpg
Removed: 0052.jpg
Removed: 0053.jpg
Removed: 0054.jpg
Removed: 0055.jpg
Removed: 0056.jpg
Removed: 0057.jpg
Removed: 0058.jpg
Removed: 0059.jpg
Removed: 0060.jpg
Removed: 0061.jpg
Removed: 0062.jpg
Removed: 0063.jpg
Removed: 0064.jpg
Removed: 0065.jpg
Removed: 0066.jpg
Removed: 0067.jpg
Removed: 0068.jpg
Removed: 0069.jpg
Removed: 0070.jpg
Removed: 0071.jpg
Removed: 0072.jpg
Removed: 0

In [103]:
# Example usage:
sequence_path = 'C:/Studie/BEP/BEP_Football_tracking/hybrid_workflow/originalseq'
prompt_no = f'C:/Studie/BEP/BEP_Football_tracking/hybrid_workflow/prompts/{framecounter}.txt'

run_script(sequence_path, prompt_no)

After every iteration of samurai, add the frame counter to the numbers of txt of the inferences and save them to a sperate folder so that they are saved

In [4]:
def move_sam_to_seq(input_folder, output_folder):
    """
    Processes text files in the input folder, renames them by adding the framecounter
    to the filename, copies them to the output folder, and clears the input folder.

    Args:
        input_folder (str): Path to the input directory.
        output_folder (str): Path to the output directory.
        framecounter (int): Value to add to the integer filenames.
    """
    global framecounter
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            # Extract the integer part of the filename
            try:
                base_name = int(os.path.splitext(filename)[0])
                # Add the framecounter to the integer part
                new_name = f"{base_name + framecounter}.txt"
                # Define full paths for input and output files
                input_file = os.path.join(input_folder, filename)
                output_file = os.path.join(output_folder, new_name)
                # Copy the file to the output folder with the new name
                shutil.copy(input_file, output_file)
            except ValueError:
                print(f"Skipping file '{filename}' as it does not have an integer name.")

    # Remove all files from the input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)

    print("Added samurai annotations to sequence and cleaned temporary samurai folder")

In [7]:
outputtxtseq = 'C:/Studie/BEP/BEP_Football_tracking/hybrid_workflow/trackedseqtxt'
inputsamuraitxt = 'C:\Studie\BEP\BEP_Football_tracking\BEP_football_tracking\Samurai\correctframes'

move_sam_to_seq(inputsamuraitxt, outputtxtseq)

Added samurai annotations to sequence and cleaned temporary samurai folder


In [8]:
framecounter = 0